<a href="https://colab.research.google.com/github/nanoNNsk/python_practice/blob/main/pos_tag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sklearn_crfsuite

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.0 MB/s eta 0:00:00


In [2]:
import nltk,re,pprint
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import pprint,time
import random
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from sklearn_crfsuite import CRF
from sklearn_crfsuite import metrics
from sklearn_crfsuite import scorers
from collections import Counter

In [3]:
nltk.download('treebank')
nltk.download('universal_tagset')

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


True

In [4]:
tagged_sentences = nltk.corpus.treebank.tagged_sents(tagset='universal')

In [5]:
print("number of tag sentence ",len(tagged_sentences))
tag_words = [tup for sent in tagged_sentences for tup in sent]
print("total number of tagged words ",len(tag_words))
vocab=set([word for word,tag in tag_words])
print("total number of vocab ",len(vocab))
tags=set([tag for word,tag in tag_words])
print("total number of tags ",len(tags))

number of tag sentence  3914
total number of tagged words  100676
total number of vocab  12408
total number of tags  12


In [6]:
train_set,test_set=train_test_split(tagged_sentences,test_size=0.2,random_state=1234)
print("number of training set ",len(train_set))
print("number of test set ",len(test_set))

number of training set  3131
number of test set  783


In [16]:
def features(sentence,index):
  return {
      'is_fist_capital':int(sentence[index][0][0].isupper()),
      'is_first_word':int(index==0),
      'is_last_word':int(index==len(sentence)-1),
      'is_complete_capital':int(sentence[index].upper()==sentence[index]),
      'prev_word':'' if index==0 else sentence[index-1][0],
      'next_word':'' if index==len(sentence)-1 else sentence[index+1][0],
      'is_numeric':int(sentence[index][0].isdigit()),
      'is_alphanumeric':int(bool((re.match('^(?=.*[0-9]$)(?=.*[a-zA-Z])',sentence[index][0])))),
      'prefix_1':sentence[index][0][0],
      'prefix_2':sentence[index][0][0:2],
      'prefix_3':sentence[index][0][0:3],
      'prefix_4':sentence[index][0][0:4],
      'suffix_1':sentence[index][0][-1],
      'suffix_2':sentence[index][0][-2:],
      'suffix_3':sentence[index][0][-3:],
      'suffix_4':sentence[index][0][-4:],
      'word_has_hyphen':1 if '-' in sentence[index][0] else 0
  }

In [17]:
def untag(sentence):
  return [word for word,tag in sentence]

def prepareData(tagged_sentences):
  X,y=[],[]
  for sentences in tagged_sentences:
    X.append([features(untag(sentences),index) for index in range(len(sentences))])
    y.append([tag for word,tag in sentences])
  return X,y

In [18]:
x_train,y_train=prepareData(train_set)
x_test,y_test=prepareData(test_set)

In [19]:
x_train[0]

[{'is_fist_capital': 1,
  'is_first_word': 1,
  'is_last_word': 0,
  'is_complete_capital': 0,
  'prev_word': '',
  'next_word': 'W',
  'is_numeric': 0,
  'is_alphanumeric': 0,
  'prefix_1': 'O',
  'prefix_2': 'O',
  'prefix_3': 'O',
  'prefix_4': 'O',
  'suffix_1': 'O',
  'suffix_2': 'O',
  'suffix_3': 'O',
  'suffix_4': 'O',
  'word_has_hyphen': 0},
 {'is_fist_capital': 1,
  'is_first_word': 0,
  'is_last_word': 0,
  'is_complete_capital': 0,
  'prev_word': 'O',
  'next_word': 'S',
  'is_numeric': 0,
  'is_alphanumeric': 0,
  'prefix_1': 'W',
  'prefix_2': 'W',
  'prefix_3': 'W',
  'prefix_4': 'W',
  'suffix_1': 'W',
  'suffix_2': 'W',
  'suffix_3': 'W',
  'suffix_4': 'W',
  'word_has_hyphen': 0},
 {'is_fist_capital': 1,
  'is_first_word': 0,
  'is_last_word': 0,
  'is_complete_capital': 0,
  'prev_word': 'W',
  'next_word': 'm',
  'is_numeric': 0,
  'is_alphanumeric': 0,
  'prefix_1': 'S',
  'prefix_2': 'S',
  'prefix_3': 'S',
  'prefix_4': 'S',
  'suffix_1': 'S',
  'suffix_2': 'S',

In [20]:
y_train[0]

['ADP',
 'NOUN',
 'NOUN',
 'NOUN',
 'CONJ',
 'NOUN',
 'VERB',
 'ADP',
 'ADJ',
 'NOUN',
 '.',
 'X',
 'VERB',
 'NUM',
 'DET',
 'ADV',
 'ADV',
 'PRON',
 'VERB',
 'ADP',
 'NOUN',
 'X',
 '.']

In [21]:
crf = CRF(
    algorithm='lbfgs',
    c1=0.01,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(x_train, y_train)

CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.01, c2=0.1,
    max_iterations=100)

In [22]:
y_pred=crf.predict(x_test)

In [23]:
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=crf.classes_)

0.7171012898409489

In [24]:
y_pred_train=crf.predict(x_train)
metrics.flat_f1_score(y_train, y_pred_train, average='weighted', labels=crf.classes_)

0.729594758619358